# 5. Optional: Data Parallelism 

이 튜토리얼 에서는 `DataParallel`을 이용해 어떻게 많은 GPU들을 사용할 수 있는지 알아본다.

만든 모델을 GPU상에서 돌리기 위해서는 아래와 같이 하면 된다.

그 다음에는 모델에 사용할 데이터를 담아놓은 Tensor를 GPU로 옮겨야 한다.

단순히 `mytensor.gpu()`를 호출하는 것은 GPU에 Tensor를 복사하는 방법이 아니다. 반드시 새로운 Tensor를 만들고, 그 Tensor에 GPU에 올리는 Tensor를 할당해야 한다.

여러 개의 GPU에서 forward, backward를 하는 것은 pytorch에서 자연스럽지만, 기본적으로는 1개의 GPU에서 실행하도록 하고있다. 여러 개의 GPU를 사용하고 싶다면 아래와 같이 한다. 아래의 코드가 이번 쳅터에서 가장 중요한 부분이다.

## Imports and parameters 

pytorch 모듈을 가져오고, parameter들을 선언해보자.

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

input_size = 5
output_size = 2

batch_size = 30
data_size = 100

## Dummy DataSet

가짜 데이터셋을 랜덤함수를 이용해서 만들어보자. 이 때 반드시 `__getitem__` 메서드를 만들어야 한다.

In [2]:
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len
    
rand_loader = DataLoader(dataset=RandomDataset(input_size, 100),
                         batch_size=batch_size, shuffle=True)

## Simple Model

간단한 모델을 만들어보자. 이 모델은 입력을 받아 linear regression을 해서 출력을 만든다. `DataParallel`은 어떤 모델도 가능하다는 걸 기억해두자.

모델을 만들 때, 모델의 입력 Tensor의 크기와 출력 Tensor의 크기을 시각적으로 보기 위해 print문을 넣을 것이다. batch rank가 0인 부분을 유심히 지켜보자.

In [7]:
class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, input):
        output = self.fc(input)
        print("In Model: input size", input.size(),
              "output size", output.size())
        
        return output

## Create Model and DataParallel

이 부분이 이 쳅터의 핵심이다. 먼저, 모델 클래스의 인스턴스를 만들어서 여러 개의 GPU를 사용 가능한 컴퓨터인지 확인하고, 가능하다면 `nn.DataParallel`을 사용해 만든 모델이 여러 개의 GPU를 가지고 연산을 할 수 있게 만든다. 그 이후에 모델을 GPU상에 올릴 수 있다. (DataParallel은 GPU를 쓸 수 있는지 없는지 체크해서 GPU를 쓸 수 없을 때는 인자로 받은 모델과 똑같이 취급되고, GPU를 쓸 수 있다면 데이터를 나눠서 처리하는 모델이 된다.)

In [8]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
    
if torch.cuda.is_available():
    model.gpu() # '.cuda()' and '.gpu()' are same thing.
    
# DataParallel은 결국 CPU의 데이터를 여러 개로 쪼개서 GPU로 보내는 건데,
# 그냥 아래처럼 하나의 코드로 써야 편하다.
 
# if torch.cuda.device_count > 1 and torch.cuda.is_available():
#     model = nn.DataParallel(model).gpu()

## Run the Model 

In [9]:
for data in rand_loader:
    if torch.cuda.is_available():
        input_var = Variable(data.gpu())
    else:
        input_var = Variable(data)
        
    output = model(input_var)
    print("Outside: input size", input_var.size(),
          "output_size", output.size())

In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


## Summary 

만약에 GPU를 여러 개 사용할 수 있다면, pytorch는 자동으로 각 GPU에게 맞게 Tensor를 쪼개 mini-batch를 만들어서 나눠줄 것이다. 

DataParallel은 CPU의 데이터를 잘 나누고, 모델을 GPU 수에 맞게 복사한다. 이후 나누어진 데이터와 복사된 모델을 여러 GPU에 보내고 GPU에 존재하는 복사된 모델에게 작업 순서를 알려준다. 작업이 다 끝나면 DataParallel은 결과들을 모으고 합쳐서 사용자에게 보내줄 것이다.